In [1]:
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np

In [2]:
def load_face(filename):
    
    image = Image.open(filename)

    image = image.convert('RGB')
    
    return asarray(image)    

In [3]:
def load_faces(directory_src):
    
    faces = list()
    
    for filename in listdir(directory_src):
        
        path = directory_src + filename
        
        try:
            faces.append(load_face(path))
        except:
            print("Erro na imagem {}".format(path))
    
    return faces

In [4]:
def load_fotos(directory_src):
    
    X, y = list(), list()
    
    for subdir in listdir (directory_src):
        
        path = directory_src + subdir + '\\'
        
        if not isdir(path):
            continue
        
        faces = load_faces(path)
        
        labels = [subdir for _ in range(len(faces))]
        
        print('>Carregadas %d faces da classe: %s' % (len(faces),subdir))
        
        X.extend(faces)
        y.extend(labels)
        
    return asarray(X), asarray(y)

In [5]:
trainX, trainy = load_fotos(directory_src = "D:\\Dataset\\Faces\\validation\\")

>Carregadas 110 faces da classe: Amanda
>Carregadas 30 faces da classe: Anita


In [6]:
trainX.shape

(140, 160, 160, 3)

In [7]:
trainy.shape

(140,)

In [8]:
from tensorflow.keras.models import load_model

In [9]:
model = load_model('facenet_keras.h5')

In [10]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [11]:
def get_embedding(model, face_pixels):
    
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    
    samples = expand_dims(face_pixels, axis=0)
    
    yhat = model.predict(samples)
    
    return yhat[0]

In [12]:
newTrainX = list()

In [13]:
for face in trainX:
    embedding = get_embedding(model, face)
    newTrainX.append(embedding)

In [14]:
newTrainX = asarray(newTrainX)

In [15]:
newTrainX.shape

(140, 128)

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame(data=newTrainX)

In [18]:
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.066584,-0.707239,0.786033,-0.423050,-0.184105,0.407288,0.061387,-0.312114,-0.182460,0.001200,...,0.936886,-0.235941,0.455467,0.149411,0.266539,0.561759,0.581234,-0.195093,0.288442,0.794110
1,-0.254819,-0.408523,0.355271,-0.564262,0.618483,0.483552,-0.060033,-0.231899,0.469212,-0.152375,...,0.813817,0.305725,-0.070748,-0.134677,0.691757,0.465732,0.418443,-0.078819,0.802185,0.628235
2,-0.102015,-0.569785,0.533720,-0.605543,-0.198376,0.250765,0.048814,-0.393678,-0.034151,-0.020827,...,0.933492,-0.187691,0.423651,-0.099390,0.279595,0.528912,0.348562,0.034506,0.405587,0.557358
3,-0.245553,-0.713182,0.685209,-0.646993,0.078951,0.115771,0.085376,-0.289389,-0.172420,0.100260,...,1.109374,0.060217,0.350298,-0.161692,0.336046,0.551599,0.416115,-0.094100,0.792096,0.645674
4,-0.108225,-0.506443,0.513092,-0.556392,-0.080764,0.303344,0.041449,-0.403089,0.095454,-0.158200,...,0.886405,-0.233637,0.563553,-0.222739,0.387020,0.439624,0.226086,-0.129448,0.485906,0.590035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,-0.515415,-0.364444,0.376679,-1.169360,-0.171952,-0.082919,0.334564,1.287003,0.722706,-0.145439,...,0.955339,-0.337408,0.911319,0.387249,-0.663056,-0.548511,0.864339,-0.452718,0.657906,0.308859
136,-0.053085,-0.128445,-0.045827,-0.832958,-0.650303,0.017163,0.393169,1.088605,0.715595,-0.157572,...,0.996309,-0.159320,1.508870,0.259740,-0.853419,-0.660579,0.600529,-0.557777,0.683590,0.084083
137,0.088084,0.431689,0.516850,-0.994467,-0.214396,0.053245,0.753357,0.683375,0.752360,0.049482,...,0.899266,-0.217035,1.238264,-0.065421,-0.967712,-0.686561,0.367021,-0.701336,0.354293,0.018954
138,0.226625,-0.122337,-0.277066,-0.220689,-0.309456,0.359915,0.060279,-0.154357,-0.121863,-0.219565,...,0.443053,0.005224,0.322420,-0.605390,-0.115777,0.365695,-0.089305,-0.025769,0.487779,0.257137


In [19]:
df['target'] = trainy

In [20]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,0.066584,-0.707239,0.786033,-0.423050,-0.184105,0.407288,0.061387,-0.312114,-0.182460,0.001200,...,-0.235941,0.455467,0.149411,0.266539,0.561759,0.581234,-0.195093,0.288442,0.794110,Amanda
1,-0.254819,-0.408523,0.355271,-0.564262,0.618483,0.483552,-0.060033,-0.231899,0.469212,-0.152375,...,0.305725,-0.070748,-0.134677,0.691757,0.465732,0.418443,-0.078819,0.802185,0.628235,Amanda
2,-0.102015,-0.569785,0.533720,-0.605543,-0.198376,0.250765,0.048814,-0.393678,-0.034151,-0.020827,...,-0.187691,0.423651,-0.099390,0.279595,0.528912,0.348562,0.034506,0.405587,0.557358,Amanda
3,-0.245553,-0.713182,0.685209,-0.646993,0.078951,0.115771,0.085376,-0.289389,-0.172420,0.100260,...,0.060217,0.350298,-0.161692,0.336046,0.551599,0.416115,-0.094100,0.792096,0.645674,Amanda
4,-0.108225,-0.506443,0.513092,-0.556392,-0.080764,0.303344,0.041449,-0.403089,0.095454,-0.158200,...,-0.233637,0.563553,-0.222739,0.387020,0.439624,0.226086,-0.129448,0.485906,0.590035,Amanda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,-0.515415,-0.364444,0.376679,-1.169360,-0.171952,-0.082919,0.334564,1.287003,0.722706,-0.145439,...,-0.337408,0.911319,0.387249,-0.663056,-0.548511,0.864339,-0.452718,0.657906,0.308859,Anita
136,-0.053085,-0.128445,-0.045827,-0.832958,-0.650303,0.017163,0.393169,1.088605,0.715595,-0.157572,...,-0.159320,1.508870,0.259740,-0.853419,-0.660579,0.600529,-0.557777,0.683590,0.084083,Anita
137,0.088084,0.431689,0.516850,-0.994467,-0.214396,0.053245,0.753357,0.683375,0.752360,0.049482,...,-0.217035,1.238264,-0.065421,-0.967712,-0.686561,0.367021,-0.701336,0.354293,0.018954,Anita
138,0.226625,-0.122337,-0.277066,-0.220689,-0.309456,0.359915,0.060279,-0.154357,-0.121863,-0.219565,...,0.005224,0.322420,-0.605390,-0.115777,0.365695,-0.089305,-0.025769,0.487779,0.257137,Anita


In [21]:
df.to_csv('faces_validation.csv', index=False)